In [ ]:
# prompt: /content/202502-citibike-tripdata_1.csv,/content/202502-citibike-tripdata_2.csv,/content/202502-citibike-tripdata_3.csv combine all of them according to time and print the head

import pandas as pd

# Create an empty list to store the dataframes
dfs = []

# List of CSV files to combine
csv_files = ['/content/202502-citibike-tripdata_1.csv',
             '/content/202502-citibike-tripdata_2.csv',
             '/content/202502-citibike-tripdata_3.csv']

# Loop through each file and read it into a pandas DataFrame
for file in csv_files:
  try:
    df = pd.read_csv(file)
    dfs.append(df)
  except FileNotFoundError:
    print(f"File not found: {file}")
  except pd.errors.ParserError:
    print(f"Error parsing file: {file}")

# Check if any DataFrames were successfully read
if dfs:
    # Concatenate all dataframes in the list
    combined_df = pd.concat(dfs, ignore_index=True)

    # Assuming 'started_at' is the time column. Replace with your actual time column name
    if 'started_at' in combined_df.columns:
        combined_df['started_at'] = pd.to_datetime(combined_df['started_at'])
        combined_df = combined_df.sort_values(by='started_at')
        print(combined_df.head())
    else:
        print("Error: 'started_at' column not found in the combined DataFrame.")
else:
    print("No CSV files were successfully read.")


<ipython-input-6-12fb6f76f63b>:16: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
<ipython-input-6-12fb6f76f63b>:16: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


                 ride_id rideable_type              started_at  \
980965  60E65A0ACC648119  classic_bike 2025-01-31 09:26:50.490   
983702  331785F8690B9B94  classic_bike 2025-01-31 11:55:46.295   
973579  A42A6E93BDEDA0C5  classic_bike 2025-01-31 12:35:07.028   
978961  B17CCD0404F9F130  classic_bike 2025-01-31 12:40:45.397   
973128  250976FCA2EACB21  classic_bike 2025-01-31 13:20:56.261   

                       ended_at             start_station_name  \
980965  2025-02-01 10:26:40.096                W 15 St & 6 Ave   
983702  2025-02-01 12:55:39.749     Myrtle Ave & St Edwards St   
973579  2025-02-01 13:35:02.541  Marcus Garvey Blvd & Macon St   
978961  2025-02-01 13:40:25.865         W 76 St & Columbus Ave   
973128  2025-02-01 14:20:39.206   Bialystoker Pl & Delancey St   

       start_station_id end_station_name end_station_id  start_lat  start_lng  \
980965          5989.02              NaN            NaN  40.738046 -73.996430   
983702          4659.02              NaN    

In [ ]:
# prompt: start_station_id. print out the  total number of unique elements and then filter out by just focussing on themanhatten area by filtering out based on the latitude and longitude available for each station id\

# Assuming 'start_station_id' and 'start_lat', 'start_lng' columns exist
# Filter for stations within Manhattan (approximate bounds)
manhattan_df = combined_df[
    (combined_df['start_lat'] >= 40.70) & (combined_df['start_lat'] <= 40.86) &
    (combined_df['start_lng'] >= -74.02) & (combined_df['start_lng'] <= -73.93)
]

#Unique start station ids
unique_start_stations = combined_df['start_station_id'].nunique()

# Unique start station ids in Manhattan
unique_start_stations_manhattan = manhattan_df['start_station_id'].nunique()

print(f"Total unique start station ids: {unique_start_stations}")
print(f"Total unique start station ids in Manhattan: {unique_start_stations_manhattan}")







Total unique start station ids: 4242
Total unique start station ids in Manhattan: 1715


In [ ]:
# prompt: seperate all the manhatten stations with their respective longitued and latitude

# Group by start station ID and get the first latitude and longitude for each station
manhattan_stations = manhattan_df.groupby('start_station_id').agg({'start_lat': 'first', 'start_lng': 'first'}).reset_index()

# Print the result
manhattan_stations


,start_station_id,start_lat,start_lng
0,4818.03,40.700295,-73.950323
1,4821.03,40.700119,-73.986200
2,4821.1,40.700763,-73.988698
3,4829.01,40.700379,-73.995481
4,4832.07,40.701120,-73.930390
...,...,...,...
1710,8532.03,40.859166,-73.933872
1711,SYS016,40.709628,-73.931457
1712,SYS033,40.728487,-74.011693
1713,SYS038,40.709306,-73.931175


In [ ]:
# prompt: downlaod this dataframe

from google.colab import files
manhattan_stations.to_csv('manhattan_stations.csv', encoding = 'utf-8-sig')
files.download('manhattan_stations.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import requests
import time

def get_urban_features(lat, lon, radius=500):
    overpass_url = "http://overpass-api.de/api/interpreter"

    query = f"""
    [out:json];
    (
      node["leisure"="park"](around:{radius},{lat},{lon});
      node["amenity"="cinema"](around:{radius},{lat},{lon});
      node["shop"="mall"](around:{radius},{lat},{lon});
    );
    out count;
    """
    try:
        response = requests.get(overpass_url, params={'data': query})
        data = response.json()
        count = len(data['elements'])
        return count
    except:
        return 0

# Add feature count column to your DataFrame
manhattan_stations['urban_feature_count'] = 0

for i, row in manhattan_stations.iterrows():
    lat = row['start_lat']
    lon = row['start_lng']
    count = get_urban_features(lat, lon)
    manhattan_stations.at[i, 'urban_feature_count'] = count
    time.sleep(1)  # Respect Overpass API rate limits


In [ ]:
import requests
import time
import pandas as pd
from tqdm import tqdm

# Initialize feature columns
manhattan_stations['num_parks'] = 0
manhattan_stations['num_malls'] = 0
manhattan_stations['num_theaters'] = 0

# Function to count features from OpenStreetMap
def count_feature(lat, lon, key, value, radius=500):
    overpass_url = "http://overpass-api.de/api/interpreter"
    query = f"""
    [out:json];
    (
      node["{key}"="{value}"](around:{radius},{lat},{lon});
    );
    out;
    """
    try:
        response = requests.get(overpass_url, params={'data': query})
        data = response.json()
        return len(data['elements'])
    except:
        return 0

# Loop with tqdm progress bar
for i, row in tqdm(manhattan_stations.iterrows(), total=manhattan_stations.shape[0], desc="Processing stations"):
    lat = row['start_lat']
    lon = row['start_lng']

    manhattan_stations.at[i, 'num_parks'] = count_feature(lat, lon, "leisure", "park")
    manhattan_stations.at[i, 'num_malls'] = count_feature(lat, lon, "shop", "mall")
    manhattan_stations.at[i, 'num_theaters'] = count_feature(lat, lon, "amenity", "cinema")

    time.sleep(1)  # Respect Overpass API rate limit


Processing stations: 100%|██████████| 1715/1715 [2:26:13<00:00,  5.12s/it]


In [ ]:
# Mount Google Drive if using Colab
from google.colab import drive
drive.mount('/content/drive')

# Save the file
manhattan_stations.to_csv('/content/drive/MyDrive/manhattan_stations_enriched.csv', index=False)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Save the enriched dataframe to a CSV file
manhattan_stations.to_csv('manhattan_stations_enriched.csv', index=False)

# If you are using Google Colab and want to save it to Google Drive:
from google.colab import drive
drive.mount('/content/drive')

# Save the file in your Google Drive
manhattan_stations.to_csv('/content/drive/MyDrive/manhattan_stations_enriched.csv', index=False)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
